In [1]:
import os
import csv
import time
import datetime
import random
import json
import threading

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 5
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = 128  # LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    epsilon = 5
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self.indexFreqs = []  # 统计词空间中的词在出现在多少个review中
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 得到逆词频
        self._getWordIndexFreq(vocab, reviews)
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../word2vec/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _getWordIndexFreq(self, vocab, reviews):
        """
        统计词汇空间中各个词出现在多少个文本中
        """
        reviewDicts = [dict(zip(review, range(len(review)))) for review in reviews]
        indexFreqs = [0] * len(vocab)
        for word in vocab:
            count = 0
            for review in reviewDicts:
                if word in review:
                    count += 1
            indexFreqs[self._wordToIndex[word]] = count
        
        self.indexFreqs = indexFreqs
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (20000, 200)
train label shape: (20000, 1)
eval data shape: (5000, 200)


In [ ]:
print(data.wordEmbedding.shape)

(31983, 200)


In [ ]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [ ]:
# 构建模型
class AdversarialLSTM(object):
    """
    Text CNN 用于文本分类
    """
    def __init__(self, config, wordEmbedding, indexFreqs):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        self.config = config
        
        # 根据词的频率计算权重
        indexFreqs[0], indexFreqs[1] = 20000, 10000
        weights = tf.cast(tf.reshape(indexFreqs / tf.reduce_sum(indexFreqs), [1, len(indexFreqs)]), dtype=tf.float32)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用词频计算新的词嵌入矩阵
            normWordEmbedding = self._normalize(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec"), weights)
            
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(normWordEmbedding, self.inputX)
            
         # 计算二元交叉熵损失 
        with tf.name_scope("loss"):
            with tf.variable_scope("Bi-LSTM", reuse=None):
                self.predictions = self._Bi_LSTMAttention(self.embeddedWords)
                self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
                losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
                loss = tf.reduce_mean(losses)
        
        with tf.name_scope("perturLoss"):
            with tf.variable_scope("Bi-LSTM", reuse=True):
                perturWordEmbedding = self._addPerturbation(self.embeddedWords, loss)
                perturPredictions = self._Bi_LSTMAttention(perturWordEmbedding)
                perturLosses = tf.nn.sigmoid_cross_entropy_with_logits(logits=perturPredictions, labels=self.inputY)
                perturLoss = tf.reduce_mean(perturLosses)
        
        self.loss = loss + perturLoss
            
    def _Bi_LSTMAttention(self, embeddedWords):
        """
        Bi-LSTM + Attention 的模型结构
        """
        
        config = self.config
        
        # 定义双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
           
            # 定义前向LSTM结构
            lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=config.model.hiddenSizes, state_is_tuple=True),
                                                         output_keep_prob=self.dropoutKeepProb)
            # 定义反向LSTM结构
            lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=config.model.hiddenSizes, state_is_tuple=True),
                                                         output_keep_prob=self.dropoutKeepProb)


            # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
            # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
            # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
            outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(lstmFwCell, lstmBwCell, 
                                                                          self.embeddedWords, dtype=tf.float32,
                                                                          scope="bi-lstm")

        
        # 在Bi-LSTM+Attention的论文中，将前向和后向的输出相加
        with tf.name_scope("Attention"):
            H = outputs[0] + outputs[1]

            # 得到Attention的输出
            output = self._attention(H)
            outputSize = config.model.hiddenSizes
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")
            
        return predictions
    
    def _attention(self, H):
        """
        利用Attention机制得到句子的向量表示
        """
        # 获得最后一层LSTM的神经元数量
        hiddenSize = config.model.hiddenSizes
        
        # 初始化一个权重向量，是可训练的参数
        W = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1))
        
        # 对Bi-LSTM的输出用激活函数做非线性转换
        M = tf.tanh(H)
        
        # 对W和M做矩阵运算，W=[batch_size, time_step, hidden_size]，计算前做维度转换成[batch_size * time_step, hidden_size]
        # newM = [batch_size, time_step, 1]，每一个时间步的输出由向量转换成一个数字
        newM = tf.matmul(tf.reshape(M, [-1, hiddenSize]), tf.reshape(W, [-1, 1]))
        
        # 对newM做维度转换成[batch_size, time_step]
        restoreM = tf.reshape(newM, [-1, config.sequenceLength])
        
        # 用softmax做归一化处理[batch_size, time_step]
        self.alpha = tf.nn.softmax(restoreM)
        
        # 利用求得的alpha的值对H进行加权求和，用矩阵运算直接操作
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.reshape(self.alpha, [-1, config.sequenceLength, 1]))
        
        # 将三维压缩成二维sequeezeR=[batch_size, hidden_size]
        sequeezeR = tf.squeeze(r)
        
        sentenceRepren = tf.tanh(sequeezeR)
        
        # 对Attention的输出可以做dropout处理
        output = tf.nn.dropout(sentenceRepren, self.dropoutKeepProb)
        
        return output
    
    def _normalize(self, wordEmbedding, weights):
        """
        对word embedding 结合权重做标准化处理
        """
        
        mean = tf.matmul(weights, wordEmbedding)
        print(mean)
        powWordEmbedding = tf.pow(wordEmbedding - mean, 2.)
        
        var = tf.matmul(weights, powWordEmbedding)
        print(var)
        stddev = tf.sqrt(1e-6 + var)
        
        return (wordEmbedding - mean) / stddev
    
    def _addPerturbation(self, embedded, loss):
        """
        添加波动到word embedding
        """
        grad, = tf.gradients(
            loss,
            embedded,
            aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)
        grad = tf.stop_gradient(grad)
        perturb = self._scaleL2(grad, self.config.model.epsilon)
        return embedded + perturb
    
    def _scaleL2(self, x, norm_length):
        # shape(x) = (batch, num_timesteps, d)
        # Divide x by max(abs(x)) for a numerically stable L2 norm.
        # 2norm(x) = a * 2norm(x/a)
        # Scale over the full sequence, dims (1, 2)
        alpha = tf.reduce_max(tf.abs(x), (1, 2), keepdims=True) + 1e-12
        l2_norm = alpha * tf.sqrt(
            tf.reduce_sum(tf.pow(x / alpha, 2), (1, 2), keepdims=True) + 1e-6)
        x_unit = x / l2_norm
        return norm_length * x_unit

In [ ]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [ ]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding
indexFreqs = data.indexFreqs

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = AdversarialLSTM(config, wordEmbedding, indexFreqs)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
#         builder = tf.saved_model.builder.SavedModelBuilder("../model/Bi-LSTM/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
#                 if currentStep % config.training.checkpointEvery == 0:
#                     # 保存模型的另一种方法，保存checkpoint文件
#                     path = saver.save(sess, "../model/Bi-LSTM/model/my-model", global_step=currentStep)
#                     print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(lstm.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#          builder.save()

Tensor("embedding/MatMul:0", shape=(1, 200), dtype=float32)
Tensor("embedding/MatMul_1:0", shape=(1, 200), dtype=float32)
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/fw/lstm_cell/kernel:0/grad/hist is illegal; using Bi-LSTM/bi-lstm/fw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/fw/lstm_cell/kernel:0/grad/sparsity is illegal; using Bi-LSTM/bi-lstm/fw/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/fw/lstm_cell/bias:0/grad/hist is illegal; using Bi-LSTM/bi-lstm/fw/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/fw/lstm_cell/bias:0/grad/sparsity is illegal; using Bi-LSTM/bi-lstm/fw/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/bw/lstm_cell/kernel:0/grad/hist is illegal; using Bi-LSTM/bi-lstm/bw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name Bi-LSTM/bi-lstm/bw/lstm_cell/kernel:0/grad/sparsity is illegal; using Bi-LSTM/bi-

2019-03-06T17:26:47.219477, step: 45, loss: 0.8660566210746765, acc: 0.7578, auc: 0.8928, precision: 0.8913, recall: 0.6119
2019-03-06T17:26:47.547599, step: 46, loss: 0.8819922208786011, acc: 0.7734, auc: 0.8801, precision: 0.8444, recall: 0.6333
2019-03-06T17:26:47.872759, step: 47, loss: 0.6891520619392395, acc: 0.8281, auc: 0.9314, precision: 0.913, recall: 0.7
2019-03-06T17:26:48.202848, step: 48, loss: 0.8492707014083862, acc: 0.7969, auc: 0.8922, precision: 0.8909, recall: 0.7101
2019-03-06T17:26:48.527978, step: 49, loss: 0.953742504119873, acc: 0.7578, auc: 0.8643, precision: 0.7826, recall: 0.7714
2019-03-06T17:26:48.853109, step: 50, loss: 0.7323314547538757, acc: 0.8438, auc: 0.9168, precision: 0.8841, recall: 0.8356
2019-03-06T17:26:49.175247, step: 51, loss: 0.7496187686920166, acc: 0.8359, auc: 0.9208, precision: 0.8689, recall: 0.803
2019-03-06T17:26:49.523317, step: 52, loss: 0.8362205028533936, acc: 0.8047, auc: 0.904, precision: 0.8125, recall: 0.8
2019-03-06T17:26:4

2019-03-06T17:27:25.398083, step: 110, loss: 0.6888757944107056, acc: 0.8672, auc: 0.936, precision: 0.8772, recall: 0.8333
2019-03-06T17:27:25.727203, step: 111, loss: 0.7310185432434082, acc: 0.8359, auc: 0.9191, precision: 0.9184, recall: 0.7258
2019-03-06T17:27:26.057321, step: 112, loss: 0.4966275990009308, acc: 0.875, auc: 0.9652, precision: 0.902, recall: 0.807
2019-03-06T17:27:26.433316, step: 113, loss: 0.5852319598197937, acc: 0.875, auc: 0.9565, precision: 0.98, recall: 0.7656
2019-03-06T17:27:26.828259, step: 114, loss: 0.5968042612075806, acc: 0.8516, auc: 0.9516, precision: 0.9123, recall: 0.7879
2019-03-06T17:27:27.242152, step: 115, loss: 0.5859141945838928, acc: 0.8828, auc: 0.9524, precision: 0.9, recall: 0.8571
2019-03-06T17:27:27.575262, step: 116, loss: 0.7663302421569824, acc: 0.8438, auc: 0.9049, precision: 0.8679, recall: 0.7797
2019-03-06T17:27:27.910365, step: 117, loss: 0.7772823572158813, acc: 0.8359, auc: 0.9157, precision: 0.873, recall: 0.8088
2019-03-06T

2019-03-06T17:27:47.952777, step: 176, loss: 0.48007094860076904, acc: 0.9141, auc: 0.9686, precision: 0.9444, recall: 0.8644
2019-03-06T17:27:48.296857, step: 177, loss: 0.49129435420036316, acc: 0.9062, auc: 0.9714, precision: 0.9194, recall: 0.8906
2019-03-06T17:27:48.652905, step: 178, loss: 0.4345542788505554, acc: 0.9219, auc: 0.9783, precision: 0.9355, recall: 0.9062
2019-03-06T17:27:48.991001, step: 179, loss: 0.505531370639801, acc: 0.8828, auc: 0.9587, precision: 0.9, recall: 0.8873
2019-03-06T17:27:49.317128, step: 180, loss: 0.6076570749282837, acc: 0.8828, auc: 0.9404, precision: 0.963, recall: 0.8
2019-03-06T17:27:49.644254, step: 181, loss: 0.6867895126342773, acc: 0.8281, auc: 0.9328, precision: 0.8644, recall: 0.7846
2019-03-06T17:27:49.969384, step: 182, loss: 0.5382500290870667, acc: 0.8672, auc: 0.9616, precision: 0.8438, recall: 0.8852
2019-03-06T17:27:50.295513, step: 183, loss: 0.6421418190002441, acc: 0.875, auc: 0.9345, precision: 0.8448, recall: 0.875
2019-03-

2019-03-06T17:28:22.940733, step: 241, loss: 0.6984232664108276, acc: 0.8594, auc: 0.9272, precision: 0.9, recall: 0.8182
2019-03-06T17:28:23.263869, step: 242, loss: 0.5827597379684448, acc: 0.8516, auc: 0.9521, precision: 0.9388, recall: 0.7419
2019-03-06T17:28:23.588002, step: 243, loss: 0.7520190477371216, acc: 0.8047, auc: 0.9292, precision: 0.88, recall: 0.6984
2019-03-06T17:28:23.914130, step: 244, loss: 0.6656299829483032, acc: 0.8594, auc: 0.9356, precision: 0.9057, recall: 0.7869
2019-03-06T17:28:24.238263, step: 245, loss: 0.5376019477844238, acc: 0.8828, auc: 0.9626, precision: 0.9322, recall: 0.8333
2019-03-06T17:28:24.561399, step: 246, loss: 0.4750059247016907, acc: 0.9062, auc: 0.97, precision: 0.9492, recall: 0.8615
2019-03-06T17:28:24.889522, step: 247, loss: 0.5112015008926392, acc: 0.9062, auc: 0.9604, precision: 0.931, recall: 0.871
2019-03-06T17:28:25.211660, step: 248, loss: 0.7101033926010132, acc: 0.8438, auc: 0.9251, precision: 0.8246, recall: 0.8246
2019-03-0

...

2019-03-06T17:29:18.179795, step: 372, loss: 0.4683660864830017, acc: 0.875, auc: 0.9757, precision: 0.9833, recall: 0.7973
2019-03-06T17:29:18.503929, step: 373, loss: 0.5286277532577515, acc: 0.8984, auc: 0.9607, precision: 0.94, recall: 0.8246
2019-03-06T17:29:18.828061, step: 374, loss: 0.46485182642936707, acc: 0.9219, auc: 0.9629, precision: 0.9796, recall: 0.8421
2019-03-06T17:29:19.154189, step: 375, loss: 0.46997150778770447, acc: 0.9062, auc: 0.9668, precision: 0.9655, recall: 0.8485
2019-03-06T17:29:19.480511, step: 376, loss: 0.6291683912277222, acc: 0.875, auc: 0.9362, precision: 0.8889, recall: 0.8276
2019-03-06T17:29:19.804675, step: 377, loss: 0.5394988059997559, acc: 0.9062, auc: 0.9604, precision: 0.8806, recall: 0.9365
2019-03-06T17:29:20.129774, step: 378, loss: 0.49553459882736206, acc: 0.9219, auc: 0.969, precision: 0.9286, recall: 0.8966
2019-03-06T17:29:20.453908, step: 379, loss: 0.4738694429397583, acc: 0.8984, auc: 0.9664, precision: 0.9028, recall: 0.9155
20

2019-03-06T17:29:51.930197, step: 437, loss: 0.536287784576416, acc: 0.9062, auc: 0.9786, precision: 0.8361, recall: 0.9623
2019-03-06T17:29:52.254302, step: 438, loss: 0.5932485461235046, acc: 0.8672, auc: 0.9482, precision: 0.8621, recall: 0.8475
2019-03-06T17:29:52.577438, step: 439, loss: 0.568769097328186, acc: 0.9141, auc: 0.9534, precision: 0.9254, recall: 0.9118
2019-03-06T17:29:52.902568, step: 440, loss: 0.4197651147842407, acc: 0.8906, auc: 0.9767, precision: 0.9655, recall: 0.8235
2019-03-06T17:29:53.228696, step: 441, loss: 0.4227755069732666, acc: 0.8906, auc: 0.9851, precision: 0.9815, recall: 0.803
2019-03-06T17:29:53.556819, step: 442, loss: 0.38943707942962646, acc: 0.9062, auc: 0.9779, precision: 0.9545, recall: 0.875
2019-03-06T17:29:53.882952, step: 443, loss: 0.482318639755249, acc: 0.8906, auc: 0.9635, precision: 0.9677, recall: 0.8333
2019-03-06T17:29:54.207086, step: 444, loss: 0.6358328461647034, acc: 0.875, auc: 0.9355, precision: 0.8906, recall: 0.8636
2019-

2019-03-06T17:30:25.683187, step: 502, loss: 0.45971211791038513, acc: 0.9219, auc: 0.9683, precision: 0.9429, recall: 0.9167
2019-03-06T17:30:26.006322, step: 503, loss: 0.40945470333099365, acc: 0.9219, auc: 0.9782, precision: 0.9833, recall: 0.8676
2019-03-06T17:30:26.349471, step: 504, loss: 0.3085762858390808, acc: 0.9375, auc: 0.9902, precision: 0.913, recall: 0.9692
2019-03-06T17:30:26.674567, step: 505, loss: 0.5003207921981812, acc: 0.9141, auc: 0.965, precision: 0.9, recall: 0.9403
2019-03-06T17:30:26.997703, step: 506, loss: 0.4688001871109009, acc: 0.8828, auc: 0.9668, precision: 0.8571, recall: 0.9231
2019-03-06T17:30:27.319841, step: 507, loss: 0.40600132942199707, acc: 0.9062, auc: 0.977, precision: 0.8906, recall: 0.9194
2019-03-06T17:30:27.644972, step: 508, loss: 0.4201846420764923, acc: 0.9141, auc: 0.9718, precision: 0.9545, recall: 0.8873
2019-03-06T17:30:27.972098, step: 509, loss: 0.45093679428100586, acc: 0.9297, auc: 0.9757, precision: 0.8704, recall: 0.9592
20

2019-03-06T17:30:47.150302, step: 568, loss: 0.43865394592285156, acc: 0.9141, auc: 0.9668, precision: 0.9492, recall: 0.875
2019-03-06T17:30:47.475432, step: 569, loss: 0.4338987469673157, acc: 0.8906, auc: 0.9776, precision: 0.9667, recall: 0.8286
2019-03-06T17:30:47.799565, step: 570, loss: 0.4792579412460327, acc: 0.9062, auc: 0.9696, precision: 0.9, recall: 0.9
2019-03-06T17:30:48.125693, step: 571, loss: 0.508510172367096, acc: 0.8984, auc: 0.9567, precision: 0.9167, recall: 0.8302
2019-03-06T17:30:48.447832, step: 572, loss: 0.46247828006744385, acc: 0.9375, auc: 0.9673, precision: 0.9322, recall: 0.9322
2019-03-06T17:30:48.774957, step: 573, loss: 0.2518726587295532, acc: 0.9375, auc: 0.9929, precision: 1.0, recall: 0.8644
2019-03-06T17:30:49.096131, step: 574, loss: 0.4141191840171814, acc: 0.9453, auc: 0.9746, precision: 0.9636, recall: 0.9138
2019-03-06T17:30:49.428211, step: 575, loss: 0.3009086847305298, acc: 0.9297, auc: 0.9839, precision: 0.9483, recall: 0.9016
2019-03-0

2019-03-06T17:31:21.304264, step: 633, loss: 0.40825390815734863, acc: 0.9453, auc: 0.9726, precision: 0.9821, recall: 0.9016
2019-03-06T17:31:21.634382, step: 634, loss: 0.46332499384880066, acc: 0.9141, auc: 0.9646, precision: 0.9104, recall: 0.9242
2019-03-06T17:31:21.982451, step: 635, loss: 0.40926167368888855, acc: 0.8984, auc: 0.9763, precision: 0.9167, recall: 0.873
2019-03-06T17:31:22.341491, step: 636, loss: 0.31475770473480225, acc: 0.9453, auc: 0.988, precision: 0.9831, recall: 0.9062
2019-03-06T17:31:22.685572, step: 637, loss: 0.24524381756782532, acc: 0.9609, auc: 0.9953, precision: 0.9492, recall: 0.9655
2019-03-06T17:31:23.027656, step: 638, loss: 0.34653908014297485, acc: 0.9375, auc: 0.9819, precision: 0.9825, recall: 0.8889
2019-03-06T17:31:23.368744, step: 639, loss: 0.31638655066490173, acc: 0.9062, auc: 0.9888, precision: 0.9692, recall: 0.863
2019-03-06T17:31:23.723796, step: 640, loss: 0.5795339345932007, acc: 0.8906, auc: 0.9573, precision: 0.9636, recall: 0.8